# Colaborative filtering

Colaborative filtering is a type of recommender systems that makes recommendations (predictions) based only on collected ratings/preferences of many users. To quote Wikipedia:

> In the newer, narrower sense, collaborative filtering is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue than that of a randomly chosen person. 

### Rating matrix

The user ratings can be collected in a rating matrix

$$R_{um}$$

each row of this matrix corresponds to one user and each column to one item (movie in this example). 
 An entry $R_{um}$ gives the rating of user $u$ for item $m$. This matrix has size $N_u\times N_m$ where $N_u$ is the number of users and $N_m$ is the numbers of items. This can be a very large number. However ss each of the users has usually rated a (very) small sample of items this matrix is sparse! In case of e.g. Netflix it has only about $1\%$ of filled entries.  

The aim of the recommender system is to predict the remaining $99\%$ of entries :) 

### Matrix Factorisation

One approach to this problem is to reduce the  rank of the rating matrix by representing it as a product of two matrices:

$$\hat{R}_{um} = U_{uk} M_{mk},\quad \hat{R}=U\cdot M^T$$

In this formulation each entry of the rating matrix $\hat R$ is a dot product of a row of users matrix $U$ and a row of movies matrix $M$. We can view each row of movie matrix as a set of _features_ and each row of the users matrix as _preferences_ for each of the features.

The number of features $K$ is a hyperparameter that we have to choose somehow. 

Reconstructing the rating matrix $\hat R$ amounts to estimating the matrices $U$ and $M$. This is alltogether  $N_u\times K + N_u\times K$ parameters. Depending on $K$ this is usually much smaller number then the number of all entries in $\hat R$.  

Estimation of $U$ and $M$ is done byminimazing the mean squared error:

$$J=\frac{1}{2}\sum_{<u,m>}\left(R_{um}-\hat{R}_{um}\right)^2$$

The sum is over a set of all non empty entries of matrix $R$ and we denote this by symbol $\langle u,m\rangle$.

Differentiating with respect to an element of the matrix $U$ we obtain:

$$\frac{\partial }{\partial {U_{uk}}} J = \sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'}\right) M_{mk}$$

In the above formula idices $u$ and $k$ are fixed and the sum runs over all the movies $m$ rated by the user $u$. 


Equating this derivative to zero gives as an equation

$$\sum_{<u,m>} M_{mk}  M_{mk'} U_{uk'} = \sum_{<u,m>}  R_{um} M_{mk}$$

which can be rewritten in vector form:

$$ \left(\sum_{<u,m>}\vec{M}_{m}  \vec{M}_{m}^T\right) \vec{U}_{u} = \sum_{<u,m>}  R_{um} \vec{M}_{m}$$

here $\vec{U}_u$ is the $u$-th row of the matrix $U$ and similarly for $\vec{M}_m$. The summation is again over all movies $m$ rated by user $u$. 

In the same way we can derive another set of equation for the rows of the matrix $M$:

$$ \left(\sum_{<u,m>}\vec{U}_{u}  \vec{U}_{u}^T\right) \vec{M}_{m} = \sum_{<u,m>}  R_{um} \vec{U}_{u}$$

Minimazing the  $J$  requires solving both of those sets of equations simultaneuosly. This cannot be done in general and requires numerical methods. One of the is the iterative merthods of alterneting least squares.

### Alternating least squares

I if we consider $M$ fixed the equiations for the rows of $U$ are ordinary linear equations. Based on this observation the method consists of filling $M$ with same random starting values and the solving for $U$

$$  \vec{U}_{u} = \left(\sum_{<u,m>}\vec{M}_{m}  \vec{M}_{m}^T\right)^{-1}\sum_{<u,m>}  R_{um} \vec{M}_{m}$$

Then keeping $U$ fixed we solve for $M$

$$ \vec{M}_{m} =\left(\sum_{<u,m>}\vec{U}_{u}  \vec{U}_{u}^T\right)  \sum_{<u,m>}  R_{um} \vec{U}_{u}$$

and the whole process is reapeted until convergence. 

## Data 

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy.sparse import csr_matrix

We will use the data from the Movielens. Please download and unpack the  ```ml-latest-small.zip``` from their [website](https://grouplens.org/datasets/movielens/latest/).

In [2]:
data = pd.read_csv('ml-latest-small/ratings.csv')

In [3]:
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
data.drop('timestamp', axis=1, inplace=True)

In [5]:
n_ratings = len(data)
data.shape

(100836, 3)

To find the number of users and movies in the data set we can use function ```nunique```:

In [6]:
n_users  = data['userId'].nunique()
n_movies = data['movieId'].nunique()
print(n_users, n_movies)

610 9724


### Problem 1

Find the size of the rating matrix, assuming $K=64$ calculate number of parameters we have to fit. 

In [7]:
n_of_parameters = (n_users + n_movies) * 64
print(n_of_parameters)

661376


In [8]:
sparse_rating_matrix = csr_matrix((data.rating.values, (data.userId.values, data.movieId.values)))

In [9]:
sparse_rating_matrix.toarray()

array([[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  4. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       ..., 
       [ 0. ,  2.5,  2. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  3. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  5. ,  0. , ...,  0. ,  0. ,  0. ]])

Compare number of non empty entries in the rating matrix to number of parameters.

In [10]:
print("NON EMPTY ENTRIES: " + str(data.shape[0]))
print("num of parameters: " + str(n_of_parameters))

NON EMPTY ENTRIES: 100836
num of parameters: 661376


Compare number of paremeters to the size of available data. 

In [11]:
ratings_shape = sparse_rating_matrix.shape
print("Available data: " + str(ratings_shape[0] * ratings_shape[1]))
print("num of parameters: " + str(n_of_parameters))

Available data: 118295710
num of parameters: 661376


In [12]:
print('Number of movies that didnt appear in the dataset: ' + str(len(np.where(~sparse_rating_matrix.toarray().any(axis=0))[0]))) 

Number of movies that didnt appear in the dataset: 183886


#### Regularisation

Those calculations show that we still have more parameters to fit then the data. This can (and will) lead to serious overfitting. One way to mitigate that is to use regularisation of the parameters by constraining their size. 

This amounts to adding another term to the cost function $J$:

$$
J=\frac{1}{2}\sum_{<u,m>}\left(R_{um}-\hat{R}_{um}\right)^2+
\frac{\lambda}{2}\left(
||U||^2 +||M||^2
\right)
$$

where the norm of the matrix $||U||^2$ denotes the sum of all the squares of the matrix  entries

$$||U||^2=\sum_{u,k}U_{ik}^2$$

The $\lambda$ is another hyperparameter to be estimated.  

After taking the gradient we obtain new sets of equations

$$ \vec{U}_{u} = \left(\sum_{<u,m>}\vec{M}_{m}  
\vec{M}_{m}^T+\lambda I\right)^{-1}
\sum_{<u,m>} R_{um}\vec{M}_{m}$$

$$ \vec{M}_{m} =\left(\sum_{<u,m>}\vec{U}_{u}  \vec{U}_{u}^T+\lambda I\right)  \sum_{<u,m>}  R_{um} \vec{U}_{u}$$

that we solve in the same iterative way. 

Before proceeding to implementation we need to split our data into train, test and validation sets. 

In [13]:
train = data.sample(frac=0.8, replace=False)

In [14]:
train.head()

,userId,movieId,rating
9326,63,6365,2.0
9122,62,139385,4.0
66478,428,2004,2.0
45949,305,34,3.0
38937,268,1210,1.0


In [15]:
test = data.drop(train.index)

In [16]:
validation = test.sample(frac=0.5, replace=False)

In [17]:
validation.head()

,userId,movieId,rating
69001,448,2269,3.0
74513,474,5944,3.5
35145,235,368,2.0
61023,396,1,5.0
35971,246,29,5.0


In [18]:
test = test.drop(validation.index)

In [19]:
test.head()

,userId,movieId,rating
0,1,1,4.0
6,1,101,5.0
12,1,223,3.0
24,1,441,4.0
39,1,673,3.0


In [20]:
n_train_users  = train.userId.nunique()
n_train_movies = train.movieId.nunique()

In [21]:
print(n_train_users, n_train_movies)

610 9012


In [22]:
n_test_users  = test.userId.nunique()
n_test_movies = test.movieId.nunique()

In [23]:
print(n_test_users, n_test_movies)

594 3690


In [24]:
n_validation_users=validation.userId.nunique()
n_validation_movies = validation.movieId.nunique()

In [25]:
print(n_validation_users, n_validation_movies)

602 3648


When implementing the ALS algorithm we will need to traverse the ratings users by user and for each user iterate over all movies that she has rated. To this end we will define an auxiliary data structur

In [26]:
by_user = train.groupby('userId').apply(lambda g: list(zip(g.movieId, g.rating)))

This is a ```pandas.Series``` indexed by the  ```userId``` that contains lists of pairs ```(movieId, rating)``` for each movie rated by this partuclar user.

Similarly we will need to traverse list of all movies and for each movie traverse list of users that have rated it, so we create another auxiliary structure. 

In [27]:
by_movie = train.groupby('movieId').apply(lambda g: list(zip(g.userId, g.rating)))

One last point to remember is that we cannot(!) use the ```userId``` to index  rows of matrix $U$! This is because the userId are not garantied to be consecutive, especially after spliting the data into three different sets. The correct way of finding the index into the U matrix for given userId is to find it's location in the series:

```ui = by_user.index.get_loc(uid)```

Same applies to movies indexes

```mi = by_movie.index.get_loc(mid)```

In [28]:
test.head()

,userId,movieId,rating
0,1,1,4.0
6,1,101,5.0
12,1,223,3.0
24,1,441,4.0
39,1,673,3.0


### Problem 2 

2.1 Implement the ALS algorithm in python. 

In [125]:
class AlternatingLeastSquares:
    def __init__(
        self,
        n_features: int = 64,
        n_iter: int = 100,
        λ: float = 4.35,
        𝜀: float = 0.05
    ):
        self.n_features = n_features
        self.n_iter = n_iter
        self.λ = λ
        self.𝜀 = 𝜀
        self.features_eye = np.eye(n_features)

    def mean_absolute_error(self, y_pred, y_true):
        nonZero = y_true != 0
        return np.mean(abs(y_pred[nonZero] - y_true[nonZero]))
    
    def mean_squared_error(self, y_pred, y_true):
        nonZero = y_true != 0
        return np.mean(np.square(y_pred[nonZero] - y_true[nonZero]))

    def normaliseRow(self, x):
        return x / sum(x)

    def generate_matrix(self, shape):
        matrix = abs(np.random.rand(shape[0], shape[1]))
        return np.apply_along_axis(self.normaliseRow, 1, matrix)
    
    def fit(self, X):
        X = X
        self.U = self.generate_matrix((X.shape[0], self.n_features))
        self.M = self.generate_matrix((X.shape[1], self.n_features))
        nonZero = X > 0
        
        for i in range(self.n_iter):
            print('Iteration', i + 1, 'of', self.n_iter)
            for i in range(X.shape[0]):
                idx = nonZero[i,:]
                self.U[i] = self._factorise(X[i,idx], self.M[idx,], self.λ, self.features_eye)

            for i in range(X.shape[1]):
                idx = nonZero[:,i]
                self.M[i] = self._factorise(X[idx,i], self.U[idx,], self.λ, self.features_eye)
        
            mse = self.mean_squared_error(self.predict(), X)
            print('Mean Squared Error:', mse)
            if mse < self.𝜀:
                print('Fitting finished, epsilon =', self.𝜀,'bound reached with MSE =', mse)
                return
            
    def predict(self):
        return self._predict(self.U, self.M)
    
    def _predict(self, U, M):
        return U.dot(M.T)
            
    def _factorise(self, r, X, λ, eye):
        a = np.dot(X.T, X) + (λ * eye)
        b = np.dot(r.T, X)
        return np.linalg.solve(a, b)

2.2 Check its predictions on the train, test and validation set. 

In [126]:
model = AlternatingLeastSquares()

mae = model.mean_absolute_error
mse = model.mean_squared_error

In [127]:
ratings_train = train.pivot_table(columns=['movieId'], index=['userId'], values='rating', dropna=False)
ratings_train = ratings_train.fillna(0)
print(ratings_train.shape)
ratings_train.values

(610, 9012)


array([[ 0. ,  0. ,  4. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       ..., 
       [ 2.5,  2. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 3. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]])

In [128]:
model.fit(ratings_train.values)

Iteration 1 of 100
Mean Squared Error: 2.27849006147
Iteration 2 of 100
Mean Squared Error: 0.162141386051
Iteration 3 of 100
Mean Squared Error: 0.120912671079
Iteration 4 of 100
Mean Squared Error: 0.111299451986
Iteration 5 of 100
Mean Squared Error: 0.107177717701
Iteration 6 of 100
Mean Squared Error: 0.104820353602
Iteration 7 of 100
Mean Squared Error: 0.103291318163
Iteration 8 of 100
Mean Squared Error: 0.102233049895
Iteration 9 of 100
Mean Squared Error: 0.101466583001
Iteration 10 of 100
Mean Squared Error: 0.100887925159
Iteration 11 of 100
Mean Squared Error: 0.100432705168
Iteration 12 of 100
Mean Squared Error: 0.10006095817
Iteration 13 of 100
Mean Squared Error: 0.099748136832
Iteration 14 of 100
Mean Squared Error: 0.0994790632575
Iteration 15 of 100
Mean Squared Error: 0.0992440442088
Iteration 16 of 100
Mean Squared Error: 0.0990365552434
Iteration 17 of 100
Mean Squared Error: 0.0988519250819
Iteration 18 of 100
Mean Squared Error: 0.0986866055237
Iteration 19 of 

In [129]:
predictions = model.predict()
print(predictions.shape)
predictions

(610, 9012)


array([[ 3.99280995,  3.77069897,  4.02054559, ...,  1.92366295,
         1.92366295,  2.27025649],
       [ 4.04312605,  3.28826066,  3.02903534, ...,  1.26565607,
         1.26565607,  1.68630846],
       [ 0.56902654,  2.14101485,  1.81015685, ...,  0.73079126,
         0.73079126,  0.86395465],
       ..., 
       [ 2.34553928,  2.15994552,  2.97451542, ...,  1.45142217,
         1.45142217,  2.12435351],
       [ 3.13879281,  3.04282366,  2.70646179, ...,  0.89499208,
         0.89499208,  1.22966197],
       [ 5.64210748,  5.24451968,  2.84431026, ...,  2.61994648,
         2.61994648,  2.60840393]])

One problem  is that the validation and test sets may contain users and/or movies that are not present in the training set. In this case the prediction is impossible. We can check this at the level of caclulating the prediction score or clean up those sets beforehand.  We will clean up the sets using an inner join on two dataframes.  

First we prepare two dataframes that contain only index which constists of unique users(movies) Id from the training sets. 

In [130]:
by_user_idx = pd.DataFrame(index = by_user.index)
by_movie_idx = pd.DataFrame(index = by_movie.index)

In [131]:
train_col_values = ratings_train.columns.values
train_idx_values = ratings_train.index.values
predictions_df = pd.DataFrame(predictions, index=train_idx_values, columns=train_col_values)

We now use this frames in join. Inner join keeps only the rows with keys apearing in both dataframes. 

In [132]:
test = test.join(by_user_idx, on='userId', how='inner')
test = test.join(by_movie_idx, on='movieId', how='inner')

In [133]:
ratings_test = test.pivot_table(columns=['movieId'], index=['userId'], values='rating', dropna=False)
ratings_test.fillna(0, inplace=True)

test_col_values = ratings_test.columns.values
test_idx_values = ratings_test.index.values

test_ratings = ratings_train[np.intersect1d(train_col_values, test_col_values)].loc[np.intersect1d(train_idx_values, test_idx_values)]
test_preds = predictions_df[np.intersect1d(train_col_values, test_col_values)].loc[np.intersect1d(train_idx_values, test_idx_values)]

In [134]:
validation = validation.join(by_user_idx, on='userId', how='inner')
validation = validation.join(by_movie_idx, on='movieId', how='inner')

validation_test = validation.pivot_table(columns=['movieId'], index=['userId'], values='rating', dropna=False)
validation_test.fillna(0, inplace=True)

validation_col_values = validation_test.columns.values
validation_idx_values = validation_test.index.values

validation_ratings = ratings_train[np.intersect1d(train_col_values, validation_col_values)].loc[np.intersect1d(train_idx_values, validation_idx_values)]
validation_preds = predictions_df[np.intersect1d(train_col_values, validation_col_values)].loc[np.intersect1d(train_idx_values, validation_idx_values)]

In [135]:
print('-----------------------')
print('Model parameters:')
print('    K -', model.n_features)
print('    λ -', model.λ)
print('    max_i -', model.n_iter)
print('-----------------------')
print('[Training set] MSE:', mse(predictions_df.values, ratings_train.values))
print('[Training set] MAE:', mae(predictions_df.values, ratings_train.values))
print('[Test set] MSE:', mse(test_preds.values, test_ratings.values))
print('[Test set] MAE:', mae(test_preds.values, test_ratings.values))
print('[Validation set] MSE:', mse(validation_preds.values, validation_ratings.values))
print('[Validation set] MAE:', mae(validation_preds.values, validation_ratings.values))

-----------------------
Model parameters:
    K - 64
    λ - 4.35
    max_i - 100
-----------------------
[Training set] MSE: 0.0963906751744
[Training set] MAE: 0.214857926503
[Test set] MSE: 0.0832501166619
[Test set] MAE: 0.1993677571
[Validation set] MSE: 0.0832395970817
[Validation set] MAE: 0.199707643283


2.3 Does the result depend on number of features (K)? How?

Large K definitely hits the performance (time complexity) 

2.4 Does the result depend on $\lambda$? How?

With lower $\lambda$ model tends to overfit the predictions

### Biases

We can extend our model by adding additional "biases" and mean:

$$\hat{R}_{um} = U_{uk} M_{mk} + b_u +c_m +\mu$$

where $\mu$ is just overall mean (on training set):

$$\mu = \frac{1}{\sum_{<u,m> } 1}\sum_{<u,m>}R_{um}$$

We can calculate gradient with respwect to biases

$$\frac{\partial }{\partial {b_{u}}} J = \sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'}- b_u -c_m -\mu\right) \cdot 1 $$

$$\sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'}- b_u -c_m -\mu\right)  $$

$$\sum_{<u,m>}  b_u =\sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'} -c_m -\mu\right)  $$

and obtain te final equations:

$$ \vec{U}_{u} = \left(\sum_{<u,m>}\vec{M}_{m}  
\vec{M}_{m}^T+\lambda I\right)^{-1}
\sum_{<u,m>} \left( R_{um} - b_u -c_m -\mu\right)\vec{M}_{m}$$

$$ \vec{M}_{m} =\left(\sum_{<u,m>}\vec{U}_{u}  \vec{U}_{u}^T+\lambda I\right)  \sum_{<u,m>} \left( R_{um} - b_u -c_m -\mu\right) \vec{U}_{u}$$

$$ b_u =\frac{1}{\lambda+\sum_{<u,m>} 1}\sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'} -c_m -\mu\right)  $$

$$ c_m =\frac{1}{\lambda+\sum_{<u,m>} 1}\sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'} -b_u -\mu\right)  $$

We solve this equations in the same way, we start from random values and  solve each equation by keeping all other varaibles fix. 

### Problem 3

3.1 Implement this version of the ALS algorithm and compare it's performance with the previous one.

In [40]:
class ALSB:
    def __init__(
        self,
        n_features: int = 64,
        n_iter: int = 3,
        λ: float = 4.44,
        𝜀: float = 0.1
    ):
        self.n_features = n_features
        self.n_iter = n_iter
        self.λ = λ
        self.𝜀 = 𝜀
        self.features_eye = np.eye(n_features)

    def mean_absolute_error(self, y_pred, y_true):
        nonZero = y_true != 0
        return np.mean(abs(y_pred[nonZero] - y_true[nonZero]))
    
    def mean_squared_error(self, y_pred, y_true):
        nonZero = y_true != 0
        return np.mean(np.square(y_pred[nonZero] - y_true[nonZero]))

    def normaliseRow(self, x):
        return x / sum(x)

    def generate_matrix(self, shape):
        matrix = abs(np.random.rand(shape[0], shape[1]))
        return np.apply_along_axis(self.normaliseRow, 1, matrix)
    
    def fit(self, X, mu):
        self.U = self.generate_matrix((X.shape[0], self.n_features))
        self.M = self.generate_matrix((X.shape[1], self.n_features))
        self.Ub = np.zeros(X.shape[0])
        self.Mb = np.zeros(X.shape[1])
        nonZero = X > 0
        
        for i in range(self.n_iter):
            print('Iteration', i + 1, 'of', self.n_iter)
            #Update biases
            #Ub
            for i, u in enumerate(train.userId.unique()):
                accum = 0
                for j, m in enumerate(train.movieId.unique()):
                    accum += X[i,j] - np.dot( self.U[i], self.M[j] ) - self.Mb[j] - mu.values[0]
                self.Ub[i] = accum/(train.movieId.nunique() + self.λ)
            #Mb
            for j, m in enumerate(train.movieId.unique()):
                accum = 0
                for i, u in enumerate(train.userId.unique()):
                    accum += X[i,j] - np.dot( self.U[i], self.M[j] ) - self.Ub[i] - mu.values[0]
                self.Mb[j] = accum/(train.userId.nunique() + self.λ)

            for i in range(X.shape[0]):
                idx = nonZero[i,:]
                self.U[i] = self._factorise(X[i,idx], self.M[idx,], self.λ, self.features_eye,
                                            self.Ub[i], self.Mb[idx,], mu.values[0])

            for i in range(X.shape[1]):
                idx = nonZero[:,i]
                self.M[i] = self._factorise(X[idx,i], self.U[idx,], self.λ, self.features_eye,
                                            self.Ub[idx,], self.Mb[i], mu.values[0])
        
            mse = self.mean_squared_error(self.predict(mu.values[0]), X)
            print('Mean Squared Error:', mse)
            if mse < self.𝜀:
                print('Fitting finished, epsilon =', self.𝜀,'bound reached with MSE =', mse)
                return
           
    def predict(self, mu):
        return self._predict(self.U, self.M, self.Ub, self.Mb, mu)
    
    def _predict(self, U, M, Ub, Mb, mu):       
        UM = U.dot(M.T)
        UM_Mb = np.ndarray(shape=(UM.shape[0],UM.shape[1]))
        for i in range(UM.shape[0]):
            UM_Mb[i] = UM[i] + Mb
            
        UM_Mb_Ub = np.ndarray(shape=(UM.shape[0],UM.shape[1]))
        for j in range(UM.shape[1]):
            UM_Mb_Ub[:,j] = UM_Mb[:,j] + Ub

        R = UM_Mb_Ub + mu
        return R  

    def _factorise(self, r, X, λ, eye, Ub, Mb, mu):
        a = np.dot(X.T, X) + (λ * eye)
        b = np.dot((r-Ub.T-Mb-mu), X)
        return np.linalg.solve(a, b)

In [41]:
model = ALSB()
#mae = model.mean_absolute_error
#mse = model.mean_squared_error

In [42]:
mu = np.mean( train[['rating']] )

ratings_train = train.pivot_table(columns=['movieId'], index=['userId'], values='rating', dropna=False)
ratings_train = ratings_train.fillna(0).values

In [43]:
model.fit(ratings_train, mu)

Iteration 1 of 3
Mean Squared Error: 2.08680558085
Iteration 2 of 3
Mean Squared Error: 0.150512036991
Iteration 3 of 3
Mean Squared Error: 0.0998288949081
Fitting finished, epsilon = 0.1 bound reached with MSE = 0.0998288949081


In [44]:
max_ind = np.argpartition(model.Mb, -10)[-10:]
min_ind = np.argpartition(model.Mb, 10)[:10]
max_movie_biases = train.movieId.unique()[max_ind]
min_movie_biases = train.movieId.unique()[min_ind]
print( max_movie_biases, "<- max indexes" )
print( min_movie_biases, "<- min indexes" )
print()
print("max biases ->", model.Mb[max_ind])
print("min biases ->", model.Mb[min_ind])

[ 33649  91542 182639  26649 103233   1984 104875   8369  27762 127096] <- max indexes
[  6598   6638   2181 179817  88672    238 117849  26750 106330  93022] <- min indexes

max biases -> [ 1.69921501  1.69921532  1.69921658  1.82573353  1.69921806  1.699221
  1.69922384  1.74978205  1.69921924  1.69921882]
min biases -> [ -8.53912904 -11.71808385  -9.13205028 -10.38720695  -8.53920612
  -8.51163295  -8.68452876 -10.31264722  -9.43245625  -8.31746031]


3.2 Using the ```movies.csv``` set associate the movies with biases. Print top ten movies with highest bias, and top ten with lowest (negative). 

In [45]:
movies = pd.read_csv('ml-latest-small/movies.csv')

In [46]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [47]:
print( "------------------------------WITH MAX BIASES-------------------------------" )
for max in max_movie_biases:
    print( movies.loc[movies['movieId'] == max] )
print("--------------------------------WITH MIN BIASES-------------------------------")
for min in min_movie_biases:
    print( movies.loc[movies['movieId'] == min] )


------------------------------WITH MAX BIASES-------------------------------
      movieId               title                genres
5906    33649  Saving Face (2004)  Comedy|Drama|Romance
      movieId                                      title  \
7770    91542  Sherlock Holmes: A Game of Shadows (2011)   

                                              genres  
7770  Action|Adventure|Comedy|Crime|Mystery|Thriller  
      movieId                                  title            genres
9667   182639  The Second Renaissance Part II (2003)  Animation|Sci-Fi
      movieId                 title                   genres
5545    26649  Lonesome Dove (1989)  Adventure|Drama|Western
      movieId                                            title  \
8195   103233  LEGO Batman: The Movie - DC Heroes Unite (2013)   

                          genres  
8195  Action|Adventure|Animation  
      movieId                                      title  genres
1461     1984  Halloween III: Season of the Witc